# DATA 586 - Project - LSTM Neuro Net

In [2]:
# import all required packages
import torch
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
import time
import random
import pandas as pd
import torchtext.legacy #used for NLP

torch.backends.cudnn.deterministic = True

In [2]:
# Set parameters for the LSTM

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 22000
LEARNING_RATE = 1e-3
BATCH_SIZE = 128
NUM_EPOCHS = 25
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 220
HIDDEN_DIM = 440
OUTPUT_DIM = 1

## Dataset

In [4]:
# Review the data set before running the neuro net
# This cell is not used when building or running the neuro net
df = pd.read_csv('HDFS_2k-parsed_3-compl.csv', delimiter = "\t")
df.head()

,"TagCode,LogLabel"
0,1 0 1 0 1 0 1 8 9 6 9 0 0 ...
1,1 1 0 1 0 1 0 0 0 0 7 9 8 ...
2,1 0 1 0 1 1 0 6 9 7 9 8 9 ...
3,1 0 1 1 0 1 0 0 0 0 8 9 7 ...
4,1 0 1 0 1 0 1 6 9 8 9 7 9 ...


In [5]:
# Delete info from previous cell
del df

In [6]:
# Define the label and text parameters

TEXT = torchtext.legacy.data.Field(sequential=True,
                  tokenize='spacy',
                  include_lengths=True) # necessary for packed_padded_sequence

LABEL = torchtext.legacy.data.LabelField(dtype=torch.float)

C:\Users\Graha\miniconda3\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [7]:
# Assign fields ('headers of csv')
# Get data from csv
fields = [('review', TEXT), ('sentiment', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
    path="HDFS_2k-parsed_3-compl.csv", format='csv',
    skip_header=True, fields=fields)

In [8]:
# Split data into training, validation and tests sets

train_data, valid_data, test_data = dataset.split(
    split_ratio=[0.75, 0.05, 0.2],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 431296
Num Valid: 115012
Num Test: 28753


In [9]:
# Build vocabulary
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 13
Number of classes: 2


In [10]:
# Display count for normal and anomaly logs
LABEL.vocab.freqs

Counter({'1': 418705, '0': 12591})

Make dataset iterators:

In [11]:
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    sort_within_batch=True, # necessary for packed_padded_sequence
    sort_key=lambda x: len(x.review),
    device=DEVICE)

In [12]:
# Test the iterators
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.review[0].size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.review[0].size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.review[0].size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break

Train
Text matrix size: torch.Size([40, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([4, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([4, 128])
Target vector size: torch.Size([128])


In [13]:
# Neuro net model
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_length):

        #[sentence len, batch size] => [sentence len, batch size, embedding size]
        embedded = self.embedding(text)
        
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length)
        
        #[sentence len, batch size, embedding size] => 
        #  output: [sentence len, batch size, hidden size]
        #  hidden: [1, batch size, hidden size]
        packed_output, (hidden, cell) = self.rnn(packed)
        
        return self.fc(hidden.squeeze(0)).view(-1) # return 0 or 1

In [14]:
INPUT_DIM = len(TEXT.vocab)

torch.manual_seed(RANDOM_SEED)
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

## Training

In [15]:
def compute_binary_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.review
            logits = model(text, text_lengths.cpu())
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.sentiment.size(0)
            correct_pred += (predicted_labels.long() == batch_data.sentiment.long()).sum()
        return correct_pred.float()/num_examples * 100

In [21]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text, text_lengths = batch_data.review
        
        ### FORWARD AND BACK PROP
        logits = model(text, text_lengths.cpu())
        cost = F.binary_cross_entropy_with_logits(logits, batch_data.sentiment)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_binary_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_binary_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_binary_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/025 | Batch 000/012 | Cost: 0.7108
training accuracy: 96.46%
valid accuracy: 96.23%
Time elapsed: 0.01 min
Epoch: 002/025 | Batch 000/012 | Cost: 0.1584
training accuracy: 96.46%
valid accuracy: 96.23%
Time elapsed: 0.02 min
Epoch: 003/025 | Batch 000/012 | Cost: 0.0908
training accuracy: 96.46%
valid accuracy: 96.23%
Time elapsed: 0.02 min
Epoch: 004/025 | Batch 000/012 | Cost: 0.1080
training accuracy: 96.46%
valid accuracy: 96.23%
Time elapsed: 0.03 min
Epoch: 005/025 | Batch 000/012 | Cost: 0.0598
training accuracy: 96.46%
valid accuracy: 96.23%
Time elapsed: 0.04 min
Epoch: 006/025 | Batch 000/012 | Cost: 0.0582
training accuracy: 96.52%
valid accuracy: 96.23%
Time elapsed: 0.04 min
Epoch: 007/025 | Batch 000/012 | Cost: 0.1627
training accuracy: 96.52%
valid accuracy: 96.23%
Time elapsed: 0.05 min
Epoch: 008/025 | Batch 000/012 | Cost: 0.1616
training accuracy: 96.52%
valid accuracy: 96.23%
Time elapsed: 0.06 min
Epoch: 009/025 | Batch 000/012 | Cost: 0.0832
training a

In [23]:
import spacy
#nlp = spacy.load('en')
nlp = spacy.load("en_core_web_sm")

def predict_sentiment(model, sentence):
    # based on:
    # https://github.com/bentrevett/pytorch-sentiment-analysis/blob/
    # master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [24]:
print('Probability positive:')
1-predict_sentiment(model, "This is such an awesome movie, I really love it!")

Probability positive:


0.5292898118495941

In [25]:
print('Probability negative:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

Probability negative:


0.475669264793396

**LSTM**

By changing the following parameters we obtained a test accuracy of 91.24%.

*   Vocabullary size: 20000 --> 22000
*   Learning rate: 1e-4 --> 1e-3
*   Embedding Dim: 128 --> 220
*   Hidden Dim: 256 --> 440

A small learning rate can cause a neuro net to get stuck in local minima. By increasing the learning we're allowing the neuro net to reach the global minima.

Increasing the number of hidden and embedded dimensions increases the complexity of the model allowing the LSTM to account for more complex models. If the number of hidden dimensions is too high it will lead to overfitting.

Increasing the vocab size allows the model to work with a larger dataset increasing the amount of data used doing the training and testing phase.

**Changes made to the GRU**

After running the GRU, without changing any parameters from the LSTM the initial testing accuracy was 91.0% which is a similar accuracy to the LSTM. After changing the epoch, learning rate and number of dimensions there were no improvements to the accuracy of the model.